# Taller: Clasificación con Diabetes Health Indicators

Fundación Universitaria Los Libertadores
MACHINE LEARNING
Noviembre de 2025
Camila Andrea Hernández González

---

## Objetivo general

Desarrollar, evaluar e interpretar un clasificador reproducible para predecir diabetes usando el conjunto de datos **Diabetes Health Indicators**, siguiendo el ciclo **CRISP-DM** de extremo a extremo, aplicando técnicas de balanceo cuando sea necesario y priorizando una métrica clave con justificación técnica.

## 1. Ciclo CRISP-DM

1. **Business understanding**
   - Problema: identificar personas con alta probabilidad de tener diabetes para priorizar prevención y diagnóstico oportuno.
   - Costo de error: es más grave no detectar un caso de diabetes (falso negativo) que alertar de más (falso positivo).

2. **Data understanding**
   - Dataset con 100 000 observaciones y 31 variables.
   - Variables sociodemográficas, estilo de vida, medidas clínicas y un diagnóstico de diabetes.

3. **Data preparation**
   - Revisión de tipos de datos y valores faltantes.
   - Separación de variable objetivo.
   - Evitar leakage removiendo `diabetes_stage` y `diabetes_risk_score`.
   - Definición de variables numéricas y categóricas.

4. **Modeling**
   - Modelo base: Regresión logística.
   - Pipeline con escalado para variables numéricas y one-hot encoding para categóricas.
   - Uso de `class_weight="balanced"` para compensar el desbalance (60% / 40%).

5. **Evaluation**
   - Validación cruzada estratificada (5-fold).
   - Métricas: recall, precision, f1, ROC-AUC, PR-AUC, balanced accuracy.
   - En test se prioriza **PR-AUC** y **recall**, por la importancia de detectar casos de diabetes.

6. **Deployment (simulado)**
   - Script `main.py`, notebook reproducible, `requirements.txt`, `README.md` y repositorio en GitHub.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_auc_score,
    average_precision_score,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)


In [ ]:
df = pd.read_csv("diabetes_dataset.csv")

print("======================================")
print("1. ESTADÍSTICA DESCRIPTIVA BÁSICA")
print("======================================\n")

print("Shape (filas, columnas):", df.shape)

print("\nColumnas:")
print(df.columns)

print("\nPrimeras filas:")
print(df.head())

print("\nTipos de datos:")
print(df.dtypes)

print("\nValores faltantes por columna:")
print(df.isna().sum())

print("\nDistribución de 'diagnosed_diabetes' (frecuencias):")
print(df["diagnosed_diabetes"].value_counts())

print("\nDistribución de 'diagnosed_diabetes' (proporciones):")
print(df["diagnosed_diabetes"].value_counts(normalize=True))

print("\nResumen estadístico de variables numéricas:")
print(df.describe().T)


In [ ]:
print("\n======================================")
print("2. DEFINICIÓN DE TARGET Y PREDICTORES")
print("======================================\n")

target = "diagnosed_diabetes"
cols_leakage = ["diabetes_stage", "diabetes_risk_score"]

X = df.drop(columns=[target] + cols_leakage)
y = df[target]

print("Shape de X (features):", X.shape)
print("Shape de y (target):", y.shape)


In [ ]:
print("\n======================================")
print("3. TRAIN-TEST SPLIT")
print("======================================\n")

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Shape X_train:", X_train.shape)
print("Shape X_test:", X_test.shape)

print("\nDistribución en y_train:")
print(y_train.value_counts(normalize=True))

print("\nDistribución en y_test:")
print(y_test.value_counts(normalize=True))


In [ ]:
print("\n======================================")
print("4. TIPOS DE VARIABLES (CATEGÓRICAS / NUMÉRICAS)")
print("======================================\n")

cat_features = X.select_dtypes(include=["object"]).columns.tolist()
num_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Variables categóricas:", cat_features)
print("\nVariables numéricas:", num_features)


In [ ]:
print("\n======================================")
print("5. GRÁFICOS DESCRIPTIVOS")
print("======================================\n")

plt.figure()
df[df["diagnosed_diabetes"] == 0]["bmi"].hist(alpha=0.6)
df[df["diagnosed_diabetes"] == 1]["bmi"].hist(alpha=0.6)
plt.xlabel("BMI")
plt.ylabel("Frecuencia")
plt.title("Distribución de BMI según diagnóstico de diabetes")
plt.legend(["No Diabetes (0)", "Diabetes (1)"])
plt.tight_layout()
plt.show()  # en notebook se ve directo

plt.figure()
df.groupby("gender")["diagnosed_diabetes"].mean().plot(kind="bar")
plt.ylabel("Proporción con diabetes")
plt.title("Proporción de diabetes por género")
plt.tight_layout()
plt.show()


In [ ]:
print("\n======================================")
print("6. PIPELINE + REGRESIÓN LOGÍSTICA")
print("======================================\n")

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ]
)

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", log_reg),
    ]
)

scoring = {
    "recall": "recall",
    "precision": "precision",
    "f1": "f1",
    "roc_auc": "roc_auc",
    "pr_auc": "average_precision",
    "bal_acc": "balanced_accuracy",
}

scores = cross_validate(
    pipeline,
    X_train,
    y_train,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

print("Resultados de validación cruzada (5-fold):\n")
for name in scoring.keys():
    valores = scores[f"test_{name}"]
    print(f"- {name}: {valores.mean():.3f} ± {valores.std():.3f}")


In [ ]:
print("\n======================================")
print("7. EVALUACIÓN EN TEST")
print("======================================\n")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

print("Reporte de clasificación (test):\n")
print(classification_report(y_test, y_pred))

roc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)

print(f"ROC-AUC en test: {roc:.3f}")
print(f"PR-AUC (Precision-Recall AUC) en test: {pr_auc:.3f}")

plt.figure()
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title("Matriz de confusión - Regresión Logística")
plt.tight_layout()
plt.show()

plt.figure()
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Curva ROC - Regresión Logística")
plt.tight_layout()
plt.show()

plt.figure()
PrecisionRecallDisplay.from_predictions(y_test, y_proba)
plt.title("Curva Precision-Recall - Regresión Logística")
plt.tight_layout()
plt.show()
